In [ ]:
# nb_process_silver_to_gold (Final Version)

# ------------------------------
# 0️⃣ Load config, logging, functions
# ------------------------------

In [1]:
%run ./nb_config


StatementMeta(, 03234b70-a632-449e-ae2a-2d50835d0265, 3, Finished, Available, Finished)

Config Loaded Successfully


In [2]:
%run ./nb_logging


StatementMeta(, 03234b70-a632-449e-ae2a-2d50835d0265, 5, Finished, Available, Finished)

2026-01-05 14:06:44 - nb_logging - INFO - Logging notebook initialized successfully.


In [3]:
%run nb_functions

StatementMeta(, 03234b70-a632-449e-ae2a-2d50835d0265, 8, Finished, Available, Finished)

In [4]:
from pyspark.sql.functions import col, avg, when, lit, current_timestamp, round
from pyspark.sql import Window
from pyspark.sql.functions import year, month, dayofmonth, hour, col, avg, round


StatementMeta(, 03234b70-a632-449e-ae2a-2d50835d0265, 9, Finished, Available, Finished)

In [5]:
# =========================================================================
# 1. SETUP & CONFIGURATION
# =========================================================================
# We use the parameters you provided to maintain consistency across the project.

# Loading Settings
is_full_load = False
FROM_DATE = '2025-01-01'
TARGET_TABLE_NAME = "SmartCity_Analysis"

# Data Source & Target Paths
_SOURCE = "smart_city"       
FILE_FORMAT = "delta"        
DOMEIN_CATEGORY = "analytics" 
_DOMEIN = "energy_reports"   

# Metadata Column Constants
PRIM_COL_NAME = "_id"
SOURCE_COL_NAME = "_source"
IMPORTDATE_COL_NAME = "_datetime_import"

logger = get_logger(__name__)
logger.info("🚀 Starting Gold Layer Process: Smart City Decision Engine")

# =========================================================================
# 2. RELATIONSHIP METADATA
# =========================================================================
# This dictionary maps how different tables relate. 
# It allows the loop in Block 4 to join tables automatically.
join_configs = {
    "energy_weather": {
        "right_dataset": "weather_config", 
        "join_key": "time", 
        "how": "left"
    },
    "energy_air_quality": {
        "right_dataset": "air_quality_config", 
        "join_key": "time", 
        "how": "left"
    }
}

# Initialize the state-management dictionary
dataframe_collection = {"prepared": {}, "gold": {}}

# =========================================================================
# 3. LOAD & PREPARE (Silver to Collection)
# =========================================================================
# We loop through the source_configs from your nb_config to load data.
for source_name, configs in source_configs.items():
    # PATH: We use the destination path from Silver (LKHS_PATH_DES)
    src_path = f"{LKHS_PATH_DES}{configs['sink']['sink_directory']}"
    
    try:
        # Load Silver Delta table
        df = spark.read.format(FILE_FORMAT).load(src_path)
        
        # 1. Selection: Use the safe select helper from your nb_functions
        if 'columns_to_select' in configs:
            df = select_columns_safe(df, configs['columns_to_select'])
            
        # 2. Time Enrichment: Standardize time for Power BI filtering
        df = df.withColumn("year", year(col("time"))) \
               .withColumn("month", month(col("time"))) \
               .withColumn("day", dayofmonth(col("time"))) \
               .withColumn("hour", hour(col("time")))
               
        dataframe_collection["prepared"][source_name] = df
        logger.info(f"📥 Prepared {source_name} for integration.")
        
    except Exception as e:
        logger.error(f"❌ Failed to load {source_name} from {src_path}: {e}")

# =========================================================================
# 4. DYNAMIC INTEGRATION (The Join Loop)
# =========================================================================
# We take 'energy_config' as the primary table and join others to it.
base_key = "energy_config"

if base_key in dataframe_collection["prepared"]:
    final_df = dataframe_collection["prepared"][base_key]
    
    for join_name, config in join_configs.items():
        right_key = config["right_dataset"]
        
        if right_key in dataframe_collection["prepared"]:
            right_df = dataframe_collection["prepared"][right_key]
            
            # ANTI-COLLISION: Remove columns already in the base table 
            # (like year, month, day) to avoid "Ambiguous Column" errors.
            cols_to_drop = [c for c in right_df.columns if c in final_df.columns and c != config["join_key"]]
            right_df = right_df.drop(*cols_to_drop)
            
            # Execute the Join based on metadata
            final_df = final_df.join(right_df, on=config["join_key"], how=config["how"])
            logger.info(f"🔗 Integrated {right_key} using {config['join_key']}")
            
    dataframe_collection["gold"]["merged_data"] = final_df
else:
    logger.error("🚫 Base dataset (energy_config) missing. Join cannot proceed.")
    raise Exception("Base dataset missing.")

# =========================================================================
# 5. SCORING & BUSINESS LOGIC
# =========================================================================
# Here we apply the Smart City specific analysis.
df_scored = dataframe_collection["gold"]["merged_data"]

# A. Trend Calculation: 3-hour moving average of price
window_spec = Window.orderBy("time").rowsBetween(-3, 0)
df_scored = df_scored.withColumn("avg_price_3h", round(avg(col("price")).over(window_spec), 4))

# B. Environmental & Economic Scoring
PM10_LIMIT = 50.0
df_scored = df_scored.withColumn(
    "clean_energy_score",
    round((col("wind_speed_10m") * 1.5) - (col("pm10") * 0.5) - (col("carbon_monoxide") * 0.1), 2)
).withColumn(
    "opportunity_status",
    when(col("pm10") > PM10_LIMIT, lit("Unsafe: Air Pollution"))
    .when((col("price") < col("avg_price_3h")) & (col("wind_speed_10m") > 15), lit("Opportunity"))
    .otherwise(lit("Normal"))
).withColumn(
    "recommendation",
    when(col("opportunity_status") == "Unsafe: Air Pollution", "STAY INDOORS: Avoid energy-intensive tasks")
    .when((col("opportunity_status") == "Opportunity") & (col("clean_energy_score") > 30), "High Priority: Cheap & Green")
    .when(col("opportunity_status") == "Opportunity", "Medium Priority: Cheap but Dirty")
    .otherwise("Wait for better conditions")
)

# C. Final Cleanup: Select only report columns (Drops metadata like _id, _source)
report_columns = [
    'time', 'year', 'month', 'day', 'hour', 
    'price', 'avg_price_3h', 'wind_speed_10m', 'direct_radiation', 
    'carbon_monoxide', 'pm10', 'clean_energy_score', 
    'opportunity_status', 'recommendation'
]
df_gold_final = select_columns_safe(df_scored, report_columns)

# =========================================================================
# 6. PERSISTENCE (Saving to Gold Table)
# =========================================================================
try:
    # Construct Gold Sink Path
    gold_sink_path = f"{PATH_GOLD}/smart_city_decisions"
    
    # Save using your nb_functions helper
    # We use 'time' as the primary key for the merge logic in Gold.
    load_data_into_delta_table(
        data=df_gold_final, 
        sink_path=gold_sink_path, 
        full_load=True, 
        primary_col_name="time"
    )

    # Register in Lakehouse Catalog for Power BI
    table_name = f"gold_smart_city_decisions"
    create_lakehouse_table(table_name, gold_sink_path)
    spark.sql(f"REFRESH TABLE {table_name}")

    logger.info(f"✅ Gold Process Complete: {table_name} is ready for Power BI.")
    logger.info(f"📊 Final Row Count: {df_gold_final.count()}")

except Exception as e:
    logger.error(f"❌ Error during Gold persistence: {str(e)}")
    raise e

StatementMeta(, 03234b70-a632-449e-ae2a-2d50835d0265, 10, Finished, Available, Finished)

2026-01-05 14:06:52 - __main__ - INFO - 🚀 Starting Gold Layer Process: Smart City Decision Engine
2026-01-05 14:06:54 - __main__ - INFO - 📥 Prepared energy_config for integration.
2026-01-05 14:06:55 - __main__ - INFO - 📥 Prepared weather_config for integration.
2026-01-05 14:06:56 - __main__ - INFO - 📥 Prepared air_quality_config for integration.
2026-01-05 14:06:56 - __main__ - INFO - 🔗 Integrated weather_config using time
2026-01-05 14:06:56 - __main__ - INFO - 🔗 Integrated air_quality_config using time
2026-01-05 14:07:20 - __main__ - INFO - ✅ Gold Process Complete: gold_smart_city_decisions is ready for Power BI.
2026-01-05 14:07:21 - __main__ - INFO - 📊 Final Row Count: 1415
